In [3]:
import numpy as np
np.set_printoptions(precision=2, suppress=True, linewidth=200)
import cvxpy as cp

In [2]:
n = 4 
m = 2
d = 5
A = np.zeros((d, n*m))
A[0, 0] = 1
A[0, 1] = -1
A[1, 1] = 1
A[1, 2] = -1
A[2, 2] = 1
A[2, 3] = -1
A[3, 4] = 1
A[3, 5] = -1
A[4, 6] = 1
A[4, 7] = -1
A

array([[ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -1.]])

In [7]:
eigs, vecs = np.linalg.eig(A.T @ A)
print(eigs)
print(vecs)

[3.41 2.   0.   0.59 2.   0.   2.   0.  ]
[[-0.27 -0.5  -0.5  -0.65  0.    0.    0.    0.  ]
 [ 0.65  0.5  -0.5  -0.27  0.    0.    0.    0.  ]
 [-0.65  0.5  -0.5   0.27  0.    0.    0.    0.  ]
 [ 0.27 -0.5  -0.5   0.65  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.71  0.71  0.    0.  ]
 [ 0.    0.    0.    0.   -0.71  0.71  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.71  0.71]
 [ 0.    0.    0.    0.    0.    0.   -0.71  0.71]]


In [8]:
A.T @ A

array([[ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  2., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  2., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  1.]])

In [10]:
np.random.seed(0)

In [17]:
scens = [2, 4, 4]
xvar = cp.Variable(2, nonneg=True)
yvar = cp.Variable((scens[0], 2), nonneg=True)
zvar = cp.Variable((scens[1], 2), nonneg=True)


In [22]:
q = []
for s in scens:
    q.append(np.random.normal(size=(s, 2))*np.array([2, 1]) + np.array([9, 8]))
q[0] = np.eye(2)+ 0.2*np.ones((2, 2))
# q[1] = np.block([np.eye(2), np.eye(2)]).T
obj = cp.Maximize(cp.sum(cp.multiply(zvar, q[2])))
print(q)

[array([[1.2, 0.2],
       [0.2, 1.2]]), array([[ 8.81,  7.91],
       [11.02,  8.93],
       [ 3.96,  7.5 ],
       [10.  ,  9.36]]), array([[10.73,  9.21],
       [ 9.9 ,  6.64],
       [ 6.  ,  8.01],
       [13.5 ,  8.19]])]


In [23]:
cons = [cp.sum(xvar) == 1]
for s in range(scens[0]):
    cons += [cp.sum(yvar[s]) == q[0][s] @ xvar]#, y[s] >= cp.multiply(q[0][s], x) - .2*np.ones(2), y[s] <= cp.multiply(q[0][s], x) + .2*np.ones(2)]
for s in range(scens[1]):
    cons += [cp.sum(zvar[s]) == q[1][s] @ yvar[s // 2]]#, z[s] >= cp.multiply(q[1][s], y[s // 2]) - .2*np.ones(2), z[s] <= cp.multiply(q[1][s], y[s // 2]) + .2*np.ones(2)]


In [24]:
prob = cp.Problem(obj, cons)
prob.solve()
print(prob.value)


281.73971959767067


In [25]:
print(xvar.value)
print(yvar.value)
print(zvar.value)

# Print dual variables
for con in cons:
    print(con.dual_value)

[1. 0.]
[[1.2 0. ]
 [0.  0.2]]
[[10.57  0.  ]
 [13.23  0.  ]
 [ 0.    1.5 ]
 [ 1.87  0.  ]]
281.73971959767067
203.72086374230295
186.37341553453564
10.734187445791722
9.902785504557599
8.010965486196769
13.498561347158347


In [125]:
for s in range(scens[1]):
    print((cp.multiply(q[1][s], y[s // 2]) - .2*np.ones(2)).value)
    print((cp.multiply(q[1][s], y[s // 2]) + .2*np.ones(2)).value)

[-0.2 -0.2]
[0.2 0.2]
[-0.2 -0.2]
[0.2 0.2]
[-0.2 -0.2]
[0.2 0.2]
[-0.2  0.8]
[0.2 1.2]


In [127]:
cp.multiply(z, q[2]).value

array([[ 0.  ,  0.  ],
       [ 0.  ,  0.  ],
       [ 0.  ,  0.  ],
       [10.18,  0.  ]])

In [11]:
def build_problem(q, warmstart, rho=1):
    x = cp.Variable((3, 2), nonneg=True)
    x.value = warmstart
    z = cp.Parameter((3, 2))
    z.value = warmstart
    cons = [cp.sum(x[0]) == 1,
            cp.sum(x[1]) == q[0] @ x[0],
            cp.sum(x[2]) == q[1] @ x[1]]
    obj = cp.Minimize(-cp.sum(x[2] @ q[2].T) + 0.5*rho*cp.norm(x - z, 'fro'))
    prob = cp.Problem(obj, cons)
    return prob, z, x


In [12]:
class finance():

    def __init__(self, q, warmstart):
        self.prob, self.z, self.x = build_problem(q, warmstart)

    def prox(self, x, t=1):
        self.z.value[:2] = x
        self.prob.solve()
        self.z.value[2:] = self.x.value[2:]
        return self.x.value[:2]

In [145]:
f1 = finance([np.array([1,0]), q[1][0], q[2][0]])

TypeError: finance.__init__() missing 1 required positional argument: 'warmstart'

In [146]:
z = np.zeros((3, 2))
f1.prox(z, 1)

array([[ 1.  ,  0.  ],
       [ 1.  ,  0.  ],
       [10.55,  0.  ]])

In [90]:
f2 = finance([np.array([0, 1]), q[1][1], q[2][2]])

In [91]:
f2.prox(z, 1)

array([[0.  , 1.  ],
       [0.  , 1.  ],
       [8.61, 0.  ]])

In [1]:
from oars import matrices

In [26]:
x = [np.zeros((2, 2)) for _ in range(4)]
v = [np.zeros((2, 2)) for _ in range(4)]
Lx = np.zeros((2, 2)) 
Wx = np.zeros((2, 2))
Zdr, Wdr = matrices.getFull(2)
Zmt, Wmt = matrices.getMT(4)
Zdr_full = {(i+ 2*k, j + 2*k): Zdr[i, j] for i in range(2) for j in range(2) for k in range(2)}
Wdr_full = {(i+ 2*k, j + 2*k): Wdr[i, j] for i in range(2) for j in range(2) for k in range(2)}
Zs = [Zmt, Zdr_full]
Ws = [Wmt, Wdr_full]
qlist = [[q[0][s//2], q[1][s], q[2][s]] for s in range(4)]
Ni = [[[],[]], [[0], [0]], [[1], []], [[0, 2], [2]]]
Mi = [[[1, 3],[1]], [[0,2], [0]], [[1, 3], [3]], [[0, 2], [2]]]

In [27]:
itrs = 1000
itr_period = max(1, itrs//10)
n = 4
alpha = 1
gamma = 0.5
debug = True
warm = np.zeros((3, 2))
resolvents = [finance(qd, warm) for qd in qlist]
for itr in range(itrs):
    for i in range(n):
        Lx[0] = sum(Zs[0][i, j] * x[j][0] for j in Ni[i][0])
        Lx[1] = sum(Zs[1][i, j] * x[j][1] for j in Ni[i][1])
        x[i] = resolvents[i].prox(v[i] - Lx, alpha)
        if itr%itr_period == 0 and debug:
            print('Iteration', itr, 'Node', i, 'x', x[i])

    for i in range(n):
        Wx[0] = sum(Ws[0][i, j] * x[j][0] for j in Mi[i][0]) + Ws[0][i, i] * x[i][0]
        Wx[1] = sum(Ws[1][i, j] * x[j][1] for j in Mi[i][1]) + Ws[1][i, i] * x[i][1]
        v[i] = v[i] + gamma*Wx
        if itr%itr_period == 0 and debug:
            print('Iteration', itr, 'Node', i, 'v', v[i])

    # print(x)
    # print(v)

Iteration 0 Node 0 x [[1.  0. ]
 [1.2 0. ]]
Iteration 0 Node 1 x [[1.  0. ]
 [1.2 0. ]]
Iteration 0 Node 2 x [[0.  1. ]
 [0.  1.2]]
Iteration 0 Node 3 x [[0.  1. ]
 [1.2 0. ]]
Iteration 0 Node 0 v [[-0.  0.]
 [-0.  0.]]
Iteration 0 Node 1 v [[ 0.5 -0.5]
 [ 0.  -0. ]]
Iteration 0 Node 2 v [[-0.5  0.5]
 [-1.2  1.2]]
Iteration 0 Node 3 v [[-0.   0. ]
 [ 1.2 -1.2]]
Iteration 100 Node 0 x [[1.  0. ]
 [1.2 0. ]]
Iteration 100 Node 1 x [[1.  0. ]
 [1.2 0. ]]
Iteration 100 Node 2 x [[0.  1. ]
 [0.  1.2]]
Iteration 100 Node 3 x [[0.  1. ]
 [1.2 0. ]]
Iteration 100 Node 0 v [[ 0. -0.]
 [ 0. -0.]]
Iteration 100 Node 1 v [[ 50.5 -50.5]
 [ -0.    0. ]]
Iteration 100 Node 2 v [[ -50.5   50.5]
 [-121.2  121.2]]
Iteration 100 Node 3 v [[   0.    -0. ]
 [ 121.2 -121.2]]
Iteration 200 Node 0 x [[1.  0. ]
 [1.2 0. ]]
Iteration 200 Node 1 x [[1.  0. ]
 [1.2 0. ]]
Iteration 200 Node 2 x [[0.  1. ]
 [0.  1.2]]
Iteration 200 Node 3 x [[0.  1. ]
 [1.2 0. ]]
Iteration 200 Node 0 v [[ 0. -0.]
 [ 0. -0.]]
Iterat

TypeError: 'NoneType' object is not subscriptable